<a href="https://colab.research.google.com/github/Zhr-Srj/TripSimAcin-AMR/blob/main/Code/LRcomparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We want to compare results of Machine learning models trained on the same dataset and tested on the same dataset, This time Logistic regression included.

This file includes results from Table 9

# Required packages

In [ ]:
pip install xgboost

In [ ]:
import csv

import numpy as np
import pandas as pd
import seaborn as sns

import random
from random import sample, shuffle

import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as lr

import xgboost

# Data preparation

## Loading train dataset

In [ ]:
with open(r'/content/drive/MyDrive/Dataset_SHAP_u.txt') as f:
    data = []
    for line in f.readlines():
        row = []
        for elem in line.split(sep=','):
            row.append(int(elem.strip()))
        data.append(row)
print(len(data))
print(len(data[0]))

7207
2996


In [ ]:
#Reading phenotypes file
with open(r'/content/drive/MyDrive/DrugLabel_SHAP_u.txt') as fy:
    labels = []
    drugs = [] #stores drug name in each row
    row = 0 # row number of each label (is needed because data is reordered while spliting)
    reader = csv.reader(fy)
    for i in reader:
        labels.append([row, int(i[1])])
        drugs.append(i[0])
        row += 1

print(len(labels))
print(len(drugs))
print(labels[:5])

7207
7207
[[0, 1], [1, 0], [2, 0], [3, 1], [4, 1]]


In [ ]:
labels = np.asarray(labels)

### A survey on data

In [ ]:
unq_drugs = set(drugs)
len(unq_drugs)

12

In [ ]:
per_drug_dic = {}
for i, drug in enumerate(drugs_unique):
  if drug not in per_drug_dic.keys():
    per_drug_dic[drug] = [0, 0]  # num of resistant, num of susceptible
  if labels[i][1] == 0:
    per_drug_dic[drug][1] += 1
  else:
    per_drug_dic[drug][0] += 1

per_drug_dic

{'ciprofloxacin': [651, 97],
 'doripenem': [51, 148],
 'gentamicin': [572, 94],
 'tetracycline': [511, 182],
 'cefotaxime': [568, 31],
 'meropenem': [166, 172],
 'imipenem': [196, 508],
 'amikacin': [300, 394],
 'levofloxacin': [459, 146],
 'ceftriaxone': [633, 34],
 'ceftazidime': [526, 105],
 'moxifloxacin': [57, 29]}

In [ ]:
keys = []
for element in data:
  if np.asarray(element[:2693]).tobytes() not in keys:
    keys.append(np.asarray(element[:2693]).tobytes())

print("Number of strains in dataset", len(keys))

Number of strains in dataset 766


In [ ]:
labels = np.asarray(labels)

## Loading strain_aside dataset

In [ ]:
f1 = open(r'/content/drive/MyDrive/Master_new_Sep/Strain_aside_Dataset_SHAP.txt')

external_test_set = []

for line in f1.readlines():
    row = []
    for elem in line.split(sep=','):
        row.append(int(elem.strip()))
    external_test_set.append(row)

print(len(external_test_set))
print(len(external_test_set[0]))

f1.close()

1371
2996


In [ ]:
#Reading phenotypes file
with open(r'/content/drive/MyDrive/Master_new_Sep/Strain_aside_drugLabel_SHAP.txt') as fy:
    external_test_labels = []
    external_test_drugs = [] #stores drug name in each row
    row = 0 # row number of each label (is needed because data is reordered while spliting)
    reader = csv.reader(fy)
    for i in reader:
        external_test_labels.append([row, int(i[1])])
        external_test_drugs.append(i[0])
        row += 1

print(len(external_test_labels))
print(len(external_test_drugs))
print(external_test_labels[:5])

1371
1371
[[0, 1], [1, 1], [2, 1], [3, 1], [4, 1]]


In [ ]:
external_test_labels = np.asarray(external_test_labels)

In [ ]:
len(set(external_test_drugs))

10

In [ ]:
set(external_test_drugs)

{'amikacin',
 'cefotaxime',
 'ceftazidime',
 'ceftriaxone',
 'ciprofloxacin',
 'gentamicin',
 'imipenem',
 'levofloxacin',
 'meropenem',
 'tetracycline'}

In [ ]:
unq_drugs = set(external_test_drugs)
len(unq_drugs)

10

In [ ]:
per_drug_dic = {}
for i, drug in enumerate(external_test_drugs):
  if drug not in per_drug_dic.keys():
    per_drug_dic[drug] = [0, 0]  # num of resistant, num of susceptible
  if external_test_labels[i][1] == 0:
    per_drug_dic[drug][1] += 1
  else:
    per_drug_dic[drug][0] += 1

per_drug_dic

{'ciprofloxacin': [211, 23],
 'gentamicin': [202, 35],
 'ceftriaxone': [206, 4],
 'amikacin': [181, 26],
 'ceftazidime': [211, 14],
 'meropenem': [75, 6],
 'levofloxacin': [66, 6],
 'imipenem': [75, 6],
 'tetracycline': [14, 0],
 'cefotaxime': [10, 0]}

# Evaluation functions

In [ ]:
def evaluation(true_labels, pred_labels):

  Dis_pred = []
  for i in pred_labels:
    if i < 0.5:
      Dis_pred.append(0)
    else:
      Dis_pred.append(1)

  acc = accuracy_score(true_labels, Dis_pred)
  auc = roc_auc_score(true_labels, Dis_pred)

  TN, FP, FN, TP = confusion_matrix(true_labels, Dis_pred, labels=[0, 1]).ravel()

  sen = TP/(TP+FN)
  spc = TN/(TN+FP)

  return acc, auc, sen, spc

# Models' training

## RF

In [ ]:
accs = []
aucs = []
sens = []
spcs = []

for i in range(10):
  model = RandomForestClassifier(n_estimators=100).fit(data, labels[:, 1])
  predict_RF = model.predict(external_test_set)
  acc, auc, sen, spc = evaluation(external_test_labels[:, 1], predict_RF)
  accs.append(acc)
  aucs.append(auc)
  sens.append(sen)
  spcs.append(spc)

In [ ]:
print('Accuracies: ', accs)
print('ROC-AUCs: ', aucs)
print("Sensitivities: ", sens)
print("Specificities: ", spcs)

print("Mean accuracy: ", sum(accs)/len(accs))
print("Mean auc: ", sum(aucs)/len(aucs))
print("Mean sensitivity: ", sum(sens)/len(sens))
print("Mean specificity: ", sum(spcs)/len(spcs))

Accuracies:  [0.8431801604668125, 0.8526622902990518, 0.8388037928519329, 0.8358862144420132, 0.8453683442742523, 0.8395331874544129, 0.8358862144420132, 0.8475565280816922, 0.8409919766593728, 0.8424507658643327]
ROC-AUCs:  [0.804826139088729, 0.7987210231814549, 0.791127098321343, 0.8045963229416466, 0.7984912070343726, 0.8028277378097521, 0.8008293365307754, 0.8109912070343726, 0.8149280575539568, 0.8232613908872902]
Sensitivities:  [0.8513189448441247, 0.8641087130295764, 0.8489208633093526, 0.8425259792166268, 0.8553157474020784, 0.8473221422861711, 0.8433253397282174, 0.8553157474020784, 0.8465227817745803, 0.8465227817745803]
Specificities:  [0.7583333333333333, 0.7333333333333333, 0.7333333333333333, 0.7666666666666667, 0.7416666666666667, 0.7583333333333333, 0.7583333333333333, 0.7666666666666667, 0.7833333333333333, 0.8]
Mean accuracy:  0.8422319474835886
Mean auc:  0.8050599520383693
Mean sensitivity:  0.8501199040767385
Mean specificity:  0.76


## SVM

In [ ]:
accs = []
aucs = []
sens = []
spcs = []

for i in range(10):

  model = SVC(kernel='poly').fit(data, labels[:, 1])
  predict_SVM = model.predict(external_test_set)

  acc, auc, sen, spc = evaluation(external_test_labels[:, 1], predict_SVM)
  accs.append(acc)
  aucs.append(auc)
  sens.append(sen)
  spcs.append(spc)

In [ ]:
print('Accuracies: ', accs)
print('ROC-AUCs: ', aucs)
print("Sensitivities: ", sens)
print("Specificities: ", spcs)

print("Mean accuracy: ", sum(accs)/len(accs))
print("Mean auc: ", sum(aucs)/len(aucs))
print("Mean sensitivity: ", sum(sens)/len(sens))
print("Mean specificity: ", sum(spcs)/len(spcs))

Accuracies:  [0.8424507658643327, 0.8424507658643327, 0.8424507658643327, 0.8424507658643327, 0.8424507658643327, 0.8424507658643327, 0.8424507658643327, 0.8424507658643327, 0.8424507658643327, 0.8424507658643327]
ROC-AUCs:  [0.7855915267785771, 0.7855915267785771, 0.7855915267785771, 0.7855915267785771, 0.7855915267785771, 0.7855915267785771, 0.7855915267785771, 0.7855915267785771, 0.7855915267785771, 0.7855915267785771]
Sensitivities:  [0.8545163868904876, 0.8545163868904876, 0.8545163868904876, 0.8545163868904876, 0.8545163868904876, 0.8545163868904876, 0.8545163868904876, 0.8545163868904876, 0.8545163868904876, 0.8545163868904876]
Specificities:  [0.7166666666666667, 0.7166666666666667, 0.7166666666666667, 0.7166666666666667, 0.7166666666666667, 0.7166666666666667, 0.7166666666666667, 0.7166666666666667, 0.7166666666666667, 0.7166666666666667]
Mean accuracy:  0.8424507658643325
Mean auc:  0.7855915267785772
Mean sensitivity:  0.8545163868904877
Mean specificity:  0.7166666666666667

## XGBoost

In [ ]:
accs = []
aucs = []
sens = []
spcs = []

for i in range(10):

  model = xgboost.XGBClassifier().fit(data, labels[:, 1])
  predict_XGB = model.predict(external_test_set)

  acc, auc, sen, spc = evaluation(external_test_labels[:, 1], predict_XGB)
  accs.append(acc)
  aucs.append(auc)
  sens.append(sen)
  spcs.append(spc)

In [ ]:
print('Accuracies: ', accs)
print('ROC-AUCs: ', aucs)
print("Sensitivities: ", sens)
print("Specificities: ", spcs)

print("Mean accuracy: ", sum(accs)/len(accs))
print("Mean auc: ", sum(aucs)/len(aucs))
print("Mean sensitivity: ", sum(sens)/len(sens))
print("Mean specificity: ", sum(spcs)/len(spcs))

Accuracies:  [0.9022611232676878, 0.9022611232676878, 0.9022611232676878, 0.9022611232676878, 0.9022611232676878, 0.9022611232676878, 0.9022611232676878, 0.9022611232676878, 0.9022611232676878, 0.9022611232676878]
ROC-AUCs:  [0.8447342126298961, 0.8447342126298961, 0.8447342126298961, 0.8447342126298961, 0.8447342126298961, 0.8447342126298961, 0.8447342126298961, 0.8447342126298961, 0.8447342126298961, 0.8447342126298961]
Sensitivities:  [0.9144684252597922, 0.9144684252597922, 0.9144684252597922, 0.9144684252597922, 0.9144684252597922, 0.9144684252597922, 0.9144684252597922, 0.9144684252597922, 0.9144684252597922, 0.9144684252597922]
Specificities:  [0.775, 0.775, 0.775, 0.775, 0.775, 0.775, 0.775, 0.775, 0.775, 0.775]
Mean accuracy:  0.902261123267688
Mean auc:  0.844734212629896
Mean sensitivity:  0.9144684252597923
Mean specificity:  0.7750000000000001


## Logistic regression

In [ ]:
accs = []
aucs = []
sens = []
spcs = []

for i in range(10):

  model = lr().fit(data, labels[:, 1])
  predict_lr = model.predict(external_test_set)

  acc, auc, sen, spc = evaluation(external_test_labels[:, 1], predict_lr)
  accs.append(acc)
  aucs.append(auc)
  sens.append(sen)
  spcs.append(spc)

In [ ]:
print('Accuracies: ', accs)
print('ROC-AUCs: ', aucs)
print("Sensitivities: ", sens)
print("Specificities: ", spcs)

print("Mean accuracy: ", sum(accs)/len(accs))
print("Mean auc: ", sum(aucs)/len(aucs))
print("Mean sensitivity: ", sum(sens)/len(sens))
print("Mean specificity: ", sum(spcs)/len(spcs))

Accuracies:  [0.8811086797957695, 0.8811086797957695, 0.8811086797957695, 0.8811086797957695, 0.8811086797957695, 0.8811086797957695, 0.8811086797957695, 0.8811086797957695, 0.8811086797957695, 0.8811086797957695]
ROC-AUCs:  [0.8105415667466026, 0.8105415667466026, 0.8105415667466026, 0.8105415667466026, 0.8105415667466026, 0.8105415667466026, 0.8105415667466026, 0.8105415667466026, 0.8105415667466026, 0.8105415667466026]
Sensitivities:  [0.8960831334932055, 0.8960831334932055, 0.8960831334932055, 0.8960831334932055, 0.8960831334932055, 0.8960831334932055, 0.8960831334932055, 0.8960831334932055, 0.8960831334932055, 0.8960831334932055]
Specificities:  [0.725, 0.725, 0.725, 0.725, 0.725, 0.725, 0.725, 0.725, 0.725, 0.725]
Mean accuracy:  0.8811086797957695
Mean auc:  0.8105415667466025
Mean sensitivity:  0.8960831334932055
Mean specificity:  0.7249999999999999


# Results

In [ ]:
models = ['RF', 'SVM', 'XGB', 'LR']
accs = [0.84, 0.84, 0.9, 0.88]
aucs = [0.8, 0.78, 0.84, 0.81]
sens = [0.85, 0.85, 0.91, 0.9]
spcs = [0.76, 0.71, 0.77, 0.72]

results = [accs, aucs, sens, spcs]

df = pd.DataFrame(results, columns=models, index=['acc', 'auc', 'sen', 'spc'])

df

,RF,SVM,XGB,LR
acc,0.84,0.84,0.90,0.88
auc,0.80,0.78,0.84,0.81
sen,0.85,0.85,0.91,0.90
spc,0.76,0.71,0.77,0.72
